In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
# import pandas as pd

import cv2
import numpy as np

class PixelAffinityDataset(torch.utils.data.Dataset):

    def __init__(self, text_file, root_dir, transform):
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
#         self.name_frame = pd.read_csv(text_file,sep=" ",usecols=range(1))
#         self.label_frame = pd.read_csv(text_file,sep=" ",usecols=range(1,2))
        name_frame, gt_frame = [], []
        list_file = './img.list'
        with open(list_file, 'r') as f:
            for line in f:
                name_frame.append(line.strip().split(' ')[0])
                gt_frame.append(line.strip().split(' ')[1])
#         print(name_frame, gt_frame)
           
        self.name_frame = name_frame
        self.gt_frame = gt_frame
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        img = cv2.imread(self.name_frame[idx])
        gt = cv2.imread(self.gt_frame[idx], -1)
        print(gt.shape, img.shape)
#         print(self.name_frame, self.gt_frame)
        if gt.shape[0] == 321:
            return torch.transpose(torch.from_numpy(img), 0, 1), torch.transpose(torch.from_numpy(gt), 0, 1)
        return torch.from_numpy(img), torch.from_numpy(gt)
# img, gt

    
transform = transforms.Compose(
[transforms.ToTensor(),
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
TrainSet = PixelAffinityDataset(text_file ='',
                                   root_dir = '', transform=transform)

TrainLoader = torch.utils.data.DataLoader(TrainSet,batch_size=8)


In [8]:
# x,y = next(iter(TrainLoader))

In [9]:
x, y = x.cuda(), y.cuda()

NameError: name 'x' is not defined

In [12]:
"""
Copyright (C) 2017 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-ND 4.0 license (https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode).
"""
%matplotlib inline

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import torch
from ERSModule import *
from network import *

# MODEL
# specify gpu id
gpu_id = 0

# configurations
conn8 = 1

# number of superpixels to be tested
nC_list = [100, 200, 300, 400, 500, 600]

# the file list of test data
img_folder = '../data/input'
img_fullpath = []
for filename in os.listdir(img_folder):
    if filename.endswith('.jpg'):
        img_fullpath.append(os.path.join(img_folder, filename))

print('Found %d images' % (len(img_fullpath)))
img_fullpath.sort()

# prepare output folders
imlog_dir = '../data/output'
if not os.path.exists(imlog_dir):
    os.makedirs(imlog_dir)

label_dir = []
for nC in nC_list:
    tmp_dir = os.path.join(imlog_dir, str(nC))
    label_dir.append(tmp_dir)
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir)

affinity_dir = os.path.join(imlog_dir, 'affinity')
if not os.path.exists(affinity_dir):
    os.makedirs(affinity_dir)


def get_weight(gt, outputs):
    h, w = gt.shape

    affinity = outputs[0].data.cpu().numpy()  # (2, 321, 481)
    affinity_list = affinity.flatten().tolist()
    sp_list = ERSWgtOnly(affinity_list, h, w, 100, conn8, 0.5)
    sp_label = np.reshape(np.asarray(sp_list), (h, w), order='C')

    weight = np.ones_like(gt, np.float32)

    # 求权重
    horizontal = (gt[:, 1:-1] != gt[:, :-2]) + (gt[:, 1:-1] != gt[:, 2:])
    horizontal = np.concatenate((np.zeros((321,1),dtype='int16'), horizontal.astype('int16'), np.zeros((321,1),dtype='int16')), axis=1)

    vertival = ((gt[1:-1] != gt[:-2]) + (gt[1:-1] != gt[2:]))
    vertival = np.concatenate((np.zeros((1,481),dtype='int16'), vertival.astype('int16'), np.zeros((1,481),dtype='int16')), axis=0)
    boundaries = horizontal + vertival

    dic = {}
    for y in range(0, h):
        for x in range(0, w):
            if boundaries[y][x]:
                if (gt[y][x], sp_label[y][x]) in dic:
                    weight[y][x] = dic[(gt[y][x], sp_label[y][x])]
                    continue
                superpixel_mask = sp_label == sp_label[y][x]
                gt_in_sp = superpixel_mask * gt
                if len(np.unique(gt_in_sp)) <= 2:
                    continue
                S = sum(sum(superpixel_mask)) * 1.
                S_G = sum(sum(gt_in_sp == gt[y][x])) * 1.
                weight[y][x] += (S - S_G) / S
                dic[(gt[y][x], sp_label[y][x])] = weight[y][x]
    return weight


def train():
#     print('=== START TESTING ===')
#     100075
    epoch = 20
    
#     for i in len(epoch):
    
    for i, img_path in enumerate(img_fullpath):
        
        filename = os.path.basename(img_path)
        basename = filename[0:-4]
#         print("%d: %s" % (i, filename))
#         image, gt = 
        if basename == '':
            image = cv2.imread(img_path)
            h, w, ch = image.shape

            input1 = image.transpose((2, 0, 1))
            input1 = np.float32(input1) / 255.0
            input1 = np.reshape(input1, [1, ch, h, w])
            input1 = torch.from_numpy(input1)  # input1.shape: (1, 3, 321, 481)

            # compute Canny edges
            edge = cv2.Canny(image, 50, 100)
            edge = 1. - np.float32(edge) / 255.
            edge = np.reshape(edge, [1, 1, h, w])
            input2 = torch.from_numpy(edge)
            inputs = torch.cat((input1, input2), 1)  # (1, 4, 321, 481)
            inputs = inputs.cuda(gpu_id, non_blocking=True)

            # inference
            out_x = model(inputs)  # (1, 1, 321, 481)
            inputs_t = torch.transpose(inputs, 2, 3)
            out_y_t = model(inputs_t)
            out_y = torch.transpose(out_y_t, 2, 3)
            outputs = torch.cat((out_x, out_y), 1) # (1, 2, 321, 481)
        
            return outputs

# gt, target, optimiser
gt = cv2.imread('../data/groundtruth/100075_1.png', -1)
# gt = cv2.imread('../data/groundtruth/12003_1.png', -1)

h, w = gt.shape

# 水平
horizontal = np.float32(gt[:,1:] == gt[:,:-1])
horizontal = np.concatenate((horizontal, np.zeros((321,1),dtype='float32')), axis=1)
input1 = np.reshape(horizontal, [1, 1, h, w])
input1 = torch.from_numpy(input1)
# 竖直
vertival = np.float32(gt[1:] == gt[:-1])
vertival = np.concatenate((vertival, np.zeros((1,481),dtype='float32')), axis=0)
input2 = np.reshape(vertival, [1, 1, h, w])
input2 = torch.from_numpy(input2)

target = torch.cat((input1, input2), 1)
target = target.cuda(gpu_id, non_blocking=True)

# model
model = PixelAffinityNet(nr_channel=128, conv1_size=7, use_canny=True)
# model.load_state_dict(torch.load('./bsds500.pkl', map_location=lambda storage, loc: storage))
# model.eval()
model.cuda(gpu_id)
optimiser = torch.optim.Adam(model.parameters())

# train loss    
for i in range(200):
    outputs = train()
    # compute superpixels
    x, y = dataloader(i)
    weight = get_weight(gt, outputs)
    criterion = nn.BCELoss(weight=torch.from_numpy(weight.astype('float32'))).cuda()
#     criterion = nn.BCELoss().cuda()
    loss = criterion(outputs, target)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    if i % 5 == 0:
        affinity = outputs[0].data.cpu().numpy()
        plt.figure()
        plt.imshow(affinity[1], cmap='gray')
        plt.show()
        print(loss.item())
    

OSError: [Errno 2] No such file or directory: '../data/input'

In [5]:
outputs = train()
affinity = outputs[0].data.cpu().numpy()
plt.figure()
plt.imshow(affinity[0], cmap='gray')
plt.show()

NameError: name 'train' is not defined

In [6]:

affinity = outputs[0].data.cpu().numpy()
output_affinity = np.uint8(255 * affinity)

plt.figure()
plt.imshow(output_affinity[1], cmap='gray')
plt.show()

NameError: name 'outputs' is not defined

In [7]:
output = np.zeros_like(image)
# save superpixel contour
np.copyto(output, image)
for y in range(0, h):
    for x in range(0, w):
        if (x < w - 1) and (sp_label[y, x] != sp_label[y, x + 1]):
            output[y, x, :] = [0, 0, 255]
        if (y < h - 1) and (sp_label[y, x] != sp_label[y + 1, x]):
            output[y, x, :] = [0, 0, 255]
plt.figure()
plt.imshow(output)
plt.show()

NameError: name 'image' is not defined